# Long-term longitudinal analysis of 4,187 participants reveals new insights into determinants of clonal hematopoiesis

In [ ]:
library(data.table)
library(dplyr)
library(stringr)
library(tidyr)
library(readxl)
library(ggplot2)
library(ggpubr)
library(cowplot)
theme_set(theme_cowplot())
### multinomial logistic regression
library(nnet)
## to plot 
library(GGally)


In [ ]:
## Mutation types:
mut_type <- function (nuc_subs) {
    muts <- nuc_subs
    types <- unlist(muts)
    types <- gsub("G>T", "C>A", types)
    types <- gsub("G>C", "C>G", types)
    types <- gsub("G>A", "C>T", types)
    types <- gsub("A>T", "T>A", types)
    types <- gsub("A>G", "T>C", types)
    types <- gsub("A>C", "T>G", types)
    return(types)
}

In [ ]:
################## inverse-normal transformation
INT_yang2012 <- function(x){
  y <- qnorm((rank(x, na.last='keep') - 0.5)/ sum(!is.na(x)))
  return(y)
}
##################

In [ ]:
## Load ARIC CHIP data

In [ ]:
setwd("/medpop/esp2/mesbah/projects/ch_progression/aric/pheno/")

## CHIP  

In [ ]:
load("Expansion_rate_input_data.01Dec2023.rda")
ls()

In [ ]:
table(aric_baseline_n_v05$GWAS_ID %in% cln_grt.vaf2.DP20_base.corrected$GWAS_ID.x, exclude=NULL)


table(aric_baseline_n_v05.noPrevCH$GWAS_ID %in% cln_grt.vaf2.DP20_base.corrected$GWAS_ID.x, exclude=NULL)

table(aric_baseline_n_v05.noPrevCH$GWAS_ID %in% cln_grt.vaf2.DP20_base.corrected_ordered$GWAS_ID.x, exclude=NULL)



#### Clones detected at baseline and follow-up visits

In [ ]:
load("/medpop/esp2/mesbah/projects/ch_progression/aric/epi/prep.vega_input.29Mar23.rda")
ls()

In [ ]:
## Get absolute max difference per sample
max_com.expansion.CH_v_b_v5_all <- com.expansion.CH_v_b_v5_all %>% 
  group_by(GWAS_ID) %>% 
  mutate(abs_dVAF = abs(dVAF)) %>%
  summarise(
    nCHIP = n(),
    maxABSdVAF = max(abs_dVAF),
    maxdVAF = max(.data$dVAF),
    mindVAF = min(.data$dVAF),
    maxGene = Gene[which.max(abs_dVAF)],
    Gene = paste(.data$Gene, collapse = ";"),
    Gene_Group = Gene_Group[which.max(abs_dVAF)],  
    VAF02_grp = paste(.data$VAF.v2, collapse = ";"),
    VAF05_grp = paste(.data$VAF.v5, collapse = ";"),
    dVAF_grp = paste(.data$dVAF, collapse = ";")
  )

head(max_com.expansion.CH_v_b_v5_all)

### Clone Status
0 = no change |
1 = Expansion |
2 = Regression 

In [ ]:
## No Change in clone
max_com.expansion.CH_v_b_v5_all$Clone_status <- ifelse((max_com.expansion.CH_v_b_v5_all$nCHIP==1 & 
          max_com.expansion.CH_v_b_v5_all$maxdVAF==0) |  (max_com.expansion.CH_v_b_v5_all$nCHIP>1 & 
                                                              max_com.expansion.CH_v_b_v5_all$maxdVAF==0),
                                                       0, 
                                                       ifelse((max_com.expansion.CH_v_b_v5_all$nCHIP==1 & 
                                                               max_com.expansion.CH_v_b_v5_all$maxdVAF>0) | 
                                                              (max_com.expansion.CH_v_b_v5_all$nCHIP>1 &   
                                                               max_com.expansion.CH_v_b_v5_all$maxdVAF>0 & 
                                                               max_com.expansion.CH_v_b_v5_all$maxABSdVAF==abs(max_com.expansion.CH_v_b_v5_all$maxdVAF) ),
                                                              1,
                                                              ifelse((max_com.expansion.CH_v_b_v5_all$nCHIP==1 & 
                                                                      max_com.expansion.CH_v_b_v5_all$maxdVAF<0) | 
                                                                     (max_com.expansion.CH_v_b_v5_all$nCHIP>1 & 
                                                                      max_com.expansion.CH_v_b_v5_all$maxdVAF<0 & 
                                                                      max_com.expansion.CH_v_b_v5_all$maxABSdVAF>abs(max_com.expansion.CH_v_b_v5_all$maxdVAF)) | 
                                                                     (max_com.expansion.CH_v_b_v5_all$nCHIP>1 & 
                                                                      max_com.expansion.CH_v_b_v5_all$maxdVAF>0 & 
                                                                      max_com.expansion.CH_v_b_v5_all$maxABSdVAF>abs(max_com.expansion.CH_v_b_v5_all$maxdVAF)),
                                                                     2, NA)))


table(max_com.expansion.CH_v_b_v5_all$Clone_status, exclude = NULL)
 


In [ ]:
table(aric_baseline_n_v05$CH_baseline==1 | aric_baseline_n_v05$CH_v05==1 , exclude = NULL)


table(aric_baseline_n_v05$GWAS_ID %in% 
      max_com.expansion.CH_v_b_v5_all$GWAS_ID[max_com.expansion.CH_v_b_v5_all$Clone_status==1], 
      exclude = NULL)


In [ ]:
## Annotate CHIP detected at either visit
aric_baseline_n_v05$CHIP_baseline_or_visit05 <- ifelse(aric_baseline_n_v05$CH_baseline==1 | 
                                                       aric_baseline_n_v05$CH_v05==1, 
                                            1, 0)

table(aric_baseline_n_v05$CHIP_baseline_or_visit05, exclude = NULL)



In [ ]:
## CHIP categories

aric_baseline_n_v05.clones <- merge(aric_baseline_n_v05, 
                                    max_com.expansion.CH_v_b_v5_all, 
                                    by="GWAS_ID", 
                                    all.x=T)

aric_baseline_n_v05.clones$Clone_status[is.na(aric_baseline_n_v05.clones$Clone_status)] <- 0

table(aric_baseline_n_v05.clones$Clone_status, exclude = NULL)

aric_baseline_n_v05.clones$nCHIP[is.na(aric_baseline_n_v05.clones$nCHIP)] <- 0 

table(aric_baseline_n_v05.clones$nCHIP)

aric_baseline_n_v05.clones$nCHIP_cat <- factor(ifelse(aric_baseline_n_v05.clones$nCHIP==0, 
                                                      "0", ifelse(aric_baseline_n_v05.clones$nCHIP==1,
                                                             "1", ifelse(aric_baseline_n_v05.clones$nCHIP==2,
                                                                         "2",ifelse(aric_baseline_n_v05.clones$nCHIP==3,
                                                                                    "3","4+")))), 
                                               level=c("0","1","2","3","4+"))

table(aric_baseline_n_v05.clones$nCHIP_cat)



### Expanded clones (dVAF>0) vs noCHIP or dVAF==0

In [ ]:
##
## Growing vs no change or no CHIP 
aric_baseline_n_v05.clones$CHIP_expanded_vs_noChange <- ifelse(aric_baseline_n_v05.clones$Clone_status==0, 
                                                                    0, 
                                                                    ifelse(aric_baseline_n_v05.clones$Clone_status==1, 
                                                                           1, NA))

table(aric_baseline_n_v05.clones$CHIP_expanded_vs_noChange, exclude=NULL)


In [ ]:
## from "2.1.3.Expansion_Rate.ipyt"

In [ ]:
ls()

## Passenger mutations

#### vaf<35%, DP<=400; no dbsnp
mpos10_45.dp20_400.vaf35
* keep variant only observed once 


In [ ]:
### Synonymous Passenger Mutations
synon_base.qcd <- fread("/medpop/esp2/mesbah/datasets/CHIP/ARIC/hiseq_vcf/baseline/all_HiSeq_baseline.mpos10_45.dp20_400.vaf35.tsv.gz", 
                        header=T)


synon_base.qcd$Sample_ID[synon_base.qcd$Sample_ID==1] <- "31684"

## 
length(unique(synon_base.qcd$Sample_ID))

####
##ARIC linker file
aric_linker <- readxl::read_excel("/medpop/esp2/mesbah/projects/ch_progression/aric/pheno/ARIC_WESFrz5_V02_CRAM_ID_lookup_20221213_GWASIDonly_ForBroad.xlsx") 
## merge
synon_base.qcd <- merge(aric_linker[,c(1,2)], 
                        synon_base.qcd, 
                        by.x = "hg38_CRAM_ID",
                        by.y="Sample_ID")

length(unique(synon_base.qcd$gwasid))

In [ ]:
# varID
synon_base.qcd$varID <- paste(synon_base.qcd$CHROM, 
                          synon_base.qcd$POS, 
                          synon_base.qcd$REF, 
                          synon_base.qcd$ALT, 
                          sep="_")

## Nucleotide Change
synon_base.qcd$nuc_subs <- paste(synon_base.qcd$REF, 
                             synon_base.qcd$ALT, 
                             sep=">")

synon_base.qcd$mut_type <- mut_type(synon_base.qcd$nuc_subs)

#
barplot(table(synon_base.qcd$nuc_subs , 
              synon_base.qcd$mut_type), 
        las=2, 
        col=factor(unique(synon_base.qcd$mut_type)))

####
synon_base.qcd <- synon_base.qcd %>% 
  filter(FORMAT== "GT:AD:AF:DP:F1R2:F2R1:SB") %>% 
  separate(GT, c("GT","AD","VAF","DP","F1R2","F2R1", "SB"), 
           ":", convert = TRUE)



## split AD
synon_base.qcd <- synon_base.qcd %>% 
  separate(AD, c("AD_REF","AD_ALT"), ",", convert = TRUE)

##
summary(synon_base.qcd$DP)

summary(synon_base.qcd$AD_ALT)

summary(synon_base.qcd$VAF)

cor(synon_base.qcd$VAF, synon_base.qcd$AD_ALT/synon_base.qcd$DP, use = "complete")

In [ ]:
names(synon_base.qcd)

### Filters:
* Additional Filter:
* "PASS"
* nchar()==1
* VAF<0.25
* AD_ALT>=3


In [ ]:

## 
# 
# 
nVar01qcd.synon_base <- as.data.frame(table(synon_base.qcd$varID), 
                                      stringsAsFactors = F)

# Filter
synon_base.qcd_pass_auto <- synon_base.qcd %>% 
filter(FILTER == "PASS" & 
       nchar(REF)==1 & 
       nchar(ALT)==1 & 
       !(CHROM %in% c("chrY", "chrX")) &
       AD_ALT>=3 & VAF<=0.25 &
       (varID %in% nVar01qcd.synon_base$Var1[nVar01qcd.synon_base$Freq==1])) %>% 
group_by(gwasid) %>% 
summarise(nSynonymous=n(),
            minVAF=min(VAF),
            maxVAF=max(VAF),
            avgVAF=mean(VAF),
            medianVAF=median(VAF),
            minAD=min(AD_ALT),
            maxAD=max(AD_ALT),
            avgAD=mean(AD_ALT),
            medianAD=median(AD_ALT),
            minDP=min(DP),
            maxDP=max(DP),
            avgDP=mean(DP),
            medianDP=median(DP),
            varid_1=paste(varID, collapse = ";"), 
            VAF_grp=paste(VAF, collapse = ";")
            )

## merge with longitudinal phenotype data
baseVAR01qcd.aric_baseline_n_v05 <- merge(aric_baseline_n_v05.clones, 
                                       synon_base.qcd_pass_auto,
                                       by.x="GWAS_ID", 
                                       by.y="gwasid",
                                       all.x=T)

##
baseVAR01qcd.aric_baseline_n_v05$nSynonymous[is.na(baseVAR01qcd.aric_baseline_n_v05$nSynonymous)] <- 0

table(baseVAR01qcd.aric_baseline_n_v05$nSynonymous>0, exclude = NULL)

baseVAR01qcd.aric_baseline_n_v05$minVAF[is.na(baseVAR01qcd.aric_baseline_n_v05$minVAF)] <- 0.001

## summary
sort(table(baseVAR01qcd.aric_baseline_n_v05$nSynonymous, exclude=NULL))

sort(table(baseVAR01qcd.aric_baseline_n_v05$nCHIP, exclude =NULL))
sort(table(baseVAR01qcd.aric_baseline_n_v05$nCHIP_cat, exclude =NULL))

nrow(baseVAR01qcd.aric_baseline_n_v05)


In [ ]:
### Summary of nSynon Mut
###
baseVAR01qcd.aric_baseline_n_v05$CHIP_status <- factor(ifelse(baseVAR01qcd.aric_baseline_n_v05$incident_CH==0 & 
                                                         baseVAR01qcd.aric_baseline_n_v05$CH_baseline==0,
                                                         "noCHIP",
                                                         ifelse(is.na(baseVAR01qcd.aric_baseline_n_v05$incident_CH),
                                                                "Prevalent", "Incident")), 
                                                  levels=c("noCHIP", "Prevalent", "Incident"))

table(baseVAR01qcd.aric_baseline_n_v05$CHIP_status, exclude=NULL)

### CHIP cat
baseVAR01qcd.aric_baseline_n_v05 %>% 
  group_by(CHIP_status) %>% 
  summarise( iqr=IQR(nSynonymous), 
             avg=mean(nSynonymous), 
             md=median(nSynonymous), 
             SD=sd(nSynonymous), 
             min=min(nSynonymous), 
             max= max(nSynonymous))

## nCHIP_cat
baseVAR01qcd.aric_baseline_n_v05 %>% 
  group_by(nCHIP_cat) %>% 
  summarise( iqr=IQR(nSynonymous), 
             avg=mean(nSynonymous), 
             md=median(nSynonymous), 
             SD=sd(nSynonymous), 
             min=min(nSynonymous), 
             max= max(nSynonymous))

## CHIP_Growth_status
baseVAR01qcd.aric_baseline_n_v05 %>% 
filter(!(is.na(CHIP_expanded_vs_noChange))) %>%
  group_by(CHIP_expanded_vs_noChange) %>% 
  summarise( iqr=IQR(nSynonymous), 
             avg=mean(nSynonymous), 
             md=median(nSynonymous), 
             SD=sd(nSynonymous), 
             min=min(nSynonymous), 
             max= max(nSynonymous))

In [ ]:
### Inverse normal transformation
baseVAR01qcd.aric_baseline_n_v05$Gene_Group[is.na(baseVAR01qcd.aric_baseline_n_v05$Gene_Group)] <- "NA"

baseVAR01qcd.aric_baseline_n_v05$INT_nSynonymous <- INT_yang2012(baseVAR01qcd.aric_baseline_n_v05$nSynonymous)

baseVAR01qcd.aric_baseline_n_v05$INT_nCHIP <- INT_yang2012(baseVAR01qcd.aric_baseline_n_v05$nCHIP)


In [ ]:
### Tansformed count data
###

### CHIP cat
baseVAR01qcd.aric_baseline_n_v05 %>% 
  group_by(CHIP_status) %>% 
  summarise( iqr=IQR(INT_nSynonymous), 
             avg=mean(INT_nSynonymous), 
             md=median(INT_nSynonymous), 
             SD=sd(INT_nSynonymous), 
             min=min(INT_nSynonymous), 
             max= max(INT_nSynonymous))

## nCHIP_cat
baseVAR01qcd.aric_baseline_n_v05 %>% 
  group_by(nCHIP_cat) %>% 
  summarise( iqr=IQR(INT_nSynonymous), 
             avg=mean(INT_nSynonymous), 
             md=median(INT_nSynonymous), 
             SD=sd(INT_nSynonymous), 
             min=min(INT_nSynonymous), 
             max= max(INT_nSynonymous))

## CHIP_Growth_status
baseVAR01qcd.aric_baseline_n_v05 %>% 
filter(!(is.na(CHIP_expanded_vs_noChange))) %>%
  group_by(CHIP_expanded_vs_noChange) %>% 
  summarise( iqr=IQR(INT_nSynonymous), 
             avg=mean(INT_nSynonymous), 
             md=median(INT_nSynonymous), 
             SD=sd(INT_nSynonymous), 
             min=min(INT_nSynonymous), 
             max= max(INT_nSynonymous))

In [ ]:
names(baseVAR01qcd.aric_baseline_n_v05)

In [ ]:

hist(baseVAR01qcd.aric_baseline_n_v05$nSynonymous, breaks=100)


### Save Data frame

In [ ]:
# save.image("/medpop/esp2/mesbah/projects/ch_progression/aric/passenger/data.passenger_hitchhiker_20240604.rda")

In [ ]:
# load("/medpop/esp2/mesbah/projects/ch_progression/aric/passenger/data.passenger_hitchhiker_20240529.rda")
load("/medpop/esp2/mesbah/projects/ch_progression/aric/passenger/data.passenger_hitchhiker_20240604.rda")

In [ ]:
getwd()

In [ ]:
### Expanded or no chamge
t.test(baseVAR01qcd.aric_baseline_n_v05$INT_nSynonymous[baseVAR01qcd.aric_baseline_n_v05$CHIP_expanded_vs_noChange==0], 
       baseVAR01qcd.aric_baseline_n_v05$INT_nSynonymous[baseVAR01qcd.aric_baseline_n_v05$CHIP_expanded_vs_noChange==1], 
            alternative="l")

wilcox.test(baseVAR01qcd.aric_baseline_n_v05$nSynonymous[baseVAR01qcd.aric_baseline_n_v05$CHIP_expanded_vs_noChange==0], 
       baseVAR01qcd.aric_baseline_n_v05$nSynonymous[baseVAR01qcd.aric_baseline_n_v05$CHIP_expanded_vs_noChange==1], 
            alternative="l")



# Regression Analyses

In [ ]:
nrow(baseVAR01qcd.aric_baseline_n_v05)

table(baseVAR01qcd.aric_baseline_n_v05$CHIP_baseline_or_visit05, exclude=NULL)

table(baseVAR01qcd.aric_baseline_n_v05$incident_CH, exclude=NULL)

table(baseVAR01qcd.aric_baseline_n_v05$CHIP_status, exclude=NULL)

table(baseVAR01qcd.aric_baseline_n_v05$CH_baseline, exclude=NULL)

table(baseVAR01qcd.aric_baseline_n_v05$CH_v05, exclude=NULL)


table(baseVAR01qcd.aric_baseline_n_v05$Clone_status, exclude=NULL)


In [ ]:
names(baseVAR01qcd.aric_baseline_n_v05)

## Model 1: Presence of CHIP vs. burden of Synonymous mutations
### 1. CHIP at either visit
### 2. Prevalent CHIP
### 3. CHIP at follow-up visit
### 4. Incident CHIP

In [ ]:

## CHIP at Baseline 
cat("Baseline:\n")
summary(glm(CH_baseline ~ INT_nSynonymous, 
            data = baseVAR01qcd.aric_baseline_n_v05, family="binomial" ))

## CHIP at Visit 05
cat("Baseline:\n")
summary(glm(CH_v05 ~ INT_nSynonymous, 
            data = baseVAR01qcd.aric_baseline_n_v05, family="binomial" ))
## CHIP only at Visit 05
cat("Incident:\n")
summary(glm(incident_CH ~ INT_nSynonymous, 
            data = baseVAR01qcd.aric_baseline_n_v05, family="binomial" ))

## expanded CHIP 
cat("Expanded:\n")
summary(glm(CHIP_expanded_vs_noChange ~ INT_nSynonymous, 
            data = baseVAR01qcd.aric_baseline_n_v05, family="binomial" ))

## CHIP any visit
cat("Either visit:\n")
summary(glm(CHIP_baseline_or_visit05 ~ INT_nSynonymous, 
            data = baseVAR01qcd.aric_baseline_n_v05, family="binomial" ))


In [ ]:
### Results
setwd("/medpop/esp2/mesbah/projects/ch_progression/aric/passenger")

# cat(gsub(pattern = ", ", replacement = ",", x = toString(
  # c("Dataset","Outcome", "Exposure","Beta", "SE", "t-stat", "P"))),
  # file = "passenger_vs_chip.multivariable_glm.20240604.csv", 
    # append = F, fill = T)

cat(gsub(pattern = ", ", replacement = ",", x = toString(
  c("Dataset","Outcome", "Exposure","Beta", "SE", "z-value", "P", "Cases", "Controls", "N"))),
  file = "passenger_vs_chip.multivariable_glm.2024Jul22.csv", append = F, fill = T)

### 1. CHIP at either visit

In [ ]:
## CHIP at either visit

## all covariates
mod1_anychip <- glm(CHIP_baseline_or_visit05 ~ INT_nSynonymous + 
            age_base + age_base_sqr + 
            Sex + race_BW  + ever_smoke + 
            INT_yang2012(bmi_base) + INT_yang2012(nonHDL_base) + INT_yang2012(hdl_base) +
            dm_126_base + htn_5_base + chd_is_base + 
    chol_med_base + Center + v2_vs_other, 
            data = baseVAR01qcd.aric_baseline_n_v05, 
                    family="binomial" )

summary(mod1_anychip)

####
# ggcoef_model(mod1_anychip, 
  #              include = dplyr::matches("INT_nSynonymous"),
   #             variable_labels = c("INT_nSynonymous" = "Passenger counts"), 
    #            show_p_values = FALSE, 
     #           signif_stars = FALSE,
      #          exponentiate = TRUE) +
# labs(title = "Passenger counts vs any CHIP", x = "OR") + 
# scale_x_continuous(expand = c(0, 0), limits = c(0.5, 2))

##
## Effect estimate
# cat( gsub(pattern = ", ", replacement = ",", x = toString(
  #    c("multivariable", "CHIP_baseline_or_visit05", "INT_nSynonymous",
   #     summary(mod1_anychip)$coefficients[1+1,1:4]) ) ),
    #  file = "passenger_vs_chip.multivariable_glm.20240604.csv", 
    #   append = T, fill = T)


      ######## addeded for case-control number 
      # Extract the data used in the model
model_data <- model.frame(mod1_anychip)

# Count the number of cases and controls
      table(model_data[[1]], exclude=NULL)
      
case_control_count <- table(model_data[[1]])

# Print number of cases and controls
cat("total N=", length(model_data[[1]]),"\n")
      
cat("CHIP_baseline_or_visit05: Number of controls:", case_control_count[1], "\n")

cat("CHIP_baseline_or_visit05: Number of cases:", case_control_count[2], "\n")
 
      
cat( gsub(pattern = ", ", replacement = ",", x = toString(
      c("multivariable", "CHIP_baseline_or_visit05", "INT_nSynonymous",
        summary(mod1_anychip)$coefficients[1+1,1:4],
       case_control_count[2], 
        case_control_count[1], 
        length(model_data[[1]]) ) )),
      file = "passenger_vs_chip.multivariable_glm.2024Jul22.csv", 
       append = T, fill = T)

In [ ]:
table(baseVAR01qcd.aric_baseline_n_v05$CH_baseline==1, 
      baseVAR01qcd.aric_baseline_n_v05$CH_v05==1, exclude = NULL)

table(baseVAR01qcd.aric_baseline_n_v05$CH_baseline==1, 
      baseVAR01qcd.aric_baseline_n_v05$CH_v05==0, exclude = NULL)


In [ ]:

### 2. Prevalent CHIP
################################
## ## Baseline visit
###############################

mod1_baseline <- glm(CH_baseline ~ INT_nSynonymous + 
            age_base + age_base_sqr + 
            Sex + race_BW  + ever_smoke + 
            INT_yang2012(bmi_base) + INT_yang2012(nonHDL_base) + INT_yang2012(hdl_base) +
            dm_126_base + htn_5_base + chd_is_base + 
    chol_med_base + Center + v2_vs_other, 
            data = baseVAR01qcd.aric_baseline_n_v05, 
                    family="binomial" )

summary(mod1_baseline)
## Effect estimate
# cat( gsub(pattern = ", ", replacement = ",", x = toString(
 #     c("multivariable", "CH_baseline", "INT_nSynonymous",
  #      summary(mod1_baseline)$coefficients[1+1,1:4]) ) ),
   #   file = "passenger_vs_chip.multivariable_glm.20240604.csv", 
    #   append = T, fill = T)
      ######## addeded for case-control number 
      # Extract the data used in the model
model_base <- model.frame(mod1_baseline)

# Count the number of cases and controls
      table(model_base[[1]], exclude=NULL)
      
case_control_count_base <- table(model_base[[1]])

# Print number of cases and controls
cat("total N=", length(model_base[[1]]),"\n")
      
cat("CH_baseline: Number of controls:", case_control_count_base[1], "\n")

cat("CH_baseline: Number of cases:", case_control_count_base[2], "\n")
 
      
cat( gsub(pattern = ", ", replacement = ",", x = toString(
      c("multivariable", "CH_baseline", "INT_nSynonymous",
        summary(mod1_baseline)$coefficients[1+1,1:4],
       case_control_count_base[2], 
        case_control_count_base[1], 
        length(model_base[[1]]) ) )),
      file = "passenger_vs_chip.multivariable_glm.2024Jul22.csv", 
       append = T, fill = T)

################################
## Follow-up visit
###############################
## 
mod1_v05 <- glm(CH_v05 ~ INT_nSynonymous + 
            age_base + age_base_sqr + 
            Sex + race_BW  + ever_smoke + 
            INT_yang2012(bmi_base) + INT_yang2012(nonHDL_base) + INT_yang2012(hdl_base) +
            dm_126_base + htn_5_base + chd_is_base + 
    chol_med_base + Center + v2_vs_other, 
            data = baseVAR01qcd.aric_baseline_n_v05, 
                    family="binomial" )

summary(mod1_v05)
## Effect estimate
# cat( gsub(pattern = ", ", replacement = ",", x = toString(
  #    c("multivariable", "CH_v05", "INT_nSynonymous",
   #     summary(mod1_v05)$coefficients[1+1,1:4]) ) ),
    #  file = "passenger_vs_chip.multivariable_glm.20240604.csv", 
     #  append = T, fill = T)

      ######## addeded for case-control number 
      # Extract the data used in the model
model_v05 <- model.frame(mod1_v05)

# Count the number of cases and controls
      table(model_v05[[1]], exclude=NULL)
      
case_control_count_v05 <- table(model_v05[[1]])

# Print number of cases and controls
cat("total N=", length(model_v05[[1]]),"\n")
      
cat("CH_v05: Number of controls:", case_control_count_v05[1], "\n")

cat("CH_v05: Number of cases:", case_control_count_v05[2], "\n")
 
      
cat( gsub(pattern = ", ", replacement = ",", x = toString(
      c("multivariable", "CH_v05", "INT_nSynonymous",
        summary(mod1_v05)$coefficients[1+1,1:4],
       case_control_count_v05[2], 
        case_control_count_v05[1], 
        length(model_v05[[1]]) ) )),
      file = "passenger_vs_chip.multivariable_glm.2024Jul22.csv", 
       append = T, fill = T)


################################
## ### Incident
###############################
mod1_incident <- glm(incident_CH ~ INT_nSynonymous + 
            age_base + age_base_sqr + 
            Sex + race_BW  + ever_smoke + 
            INT_yang2012(bmi_base) + INT_yang2012(nonHDL_base) + INT_yang2012(hdl_base) +
            dm_126_base + htn_5_base + chd_is_base + 
    chol_med_base + Center + v2_vs_other, 
            data = baseVAR01qcd.aric_baseline_n_v05, 
                    family="binomial" )

summary(mod1_incident)
## Effect estimate
# cat( gsub(pattern = ", ", replacement = ",", x = toString(
  #    c("multivariable", "incident_CH", "INT_nSynonymous",
   #     summary(mod1_incident)$coefficients[1+1,1:4]) ) ),
    #  file = "passenger_vs_chip.multivariable_glm.20240604.csv", 
     #  append = T, fill = T)


      ######## addeded for case-control number 
      # Extract the data used in the model
model_incident <- model.frame(mod1_incident)

# Count the number of cases and controls
      table(model_incident[[1]], exclude=NULL)
      
case_control_count_incident <- table(model_incident[[1]])

# Print number of cases and controls
cat("total N=", length(model_incident[[1]]),"\n")
      
cat("incident_CH: Number of controls:", case_control_count_incident[1], "\n")

cat("incident_CH: Number of cases:", case_control_count_incident[2], "\n")
 
      
cat( gsub(pattern = ", ", replacement = ",", x = toString(
      c("multivariable", "incident_CH", "INT_nSynonymous",
        summary(mod1_incident)$coefficients[1+1,1:4],
       case_control_count_incident[2], 
        case_control_count_incident[1], 
        length(model_incident[[1]]) ) )),
      file = "passenger_vs_chip.multivariable_glm.2024Jul22.csv", 
       append = T, fill = T)



## Model 3: Growth of clones vs. burden of Synonymous mutations
### Expanded clone [dVAF>0]

In [ ]:
################################
## CHIP_expanded_vs_noChange
###############################
mod1_expanded <- glm(CHIP_expanded_vs_noChange ~ INT_nSynonymous + 
            age_base + age_base_sqr + 
            Sex + race_BW  + ever_smoke + 
            INT_yang2012(bmi_base) + INT_yang2012(nonHDL_base) + INT_yang2012(hdl_base) +
            dm_126_base + htn_5_base + chd_is_base + 
    chol_med_base + Center + v2_vs_other, 
            data = baseVAR01qcd.aric_baseline_n_v05, 
                    family="binomial" )

summary(mod1_expanded)

## Effect estimate
# cat( gsub(pattern = ", ", replacement = ",", x = toString(
  #    c("multivariable", "CHIP_expanded_vs_noChange", "INT_nSynonymous",
   #     summary(mod1_expanded)$coefficients[1+1,1:4]) ) ),
    #  file = "passenger_vs_chip.multivariable_glm.20240604.csv", 
     #  append = T, fill = T)


      ######## addeded for case-control number 
      # Extract the data used in the model
model_expanded <- model.frame(mod1_expanded)

# Count the number of cases and controls
      table(model_expanded[[1]], exclude=NULL)
      
case_control_count_expanded <- table(model_expanded[[1]])

# Print number of cases and controls
cat("total N=", length(model_expanded[[1]]),"\n")
      
cat("CHIP_expanded_vs_noChange: Number of controls:", case_control_count_expanded[1], "\n")

cat("CHIP_expanded_vs_noChange: Number of cases:", case_control_count_expanded[2], "\n")
 
      
cat( gsub(pattern = ", ", replacement = ",", x = toString(
      c("multivariable", "CHIP_expanded_vs_noChange", "INT_nSynonymous",
        summary(mod1_expanded)$coefficients[1+1,1:4],
       case_control_count_expanded[2], 
        case_control_count_expanded[1], 
        length(model_expanded[[1]]) ) )),
      file = "passenger_vs_chip.multivariable_glm.2024Jul22.csv", 
       append = T, fill = T)


## Model 2. Number of CHIP vs. burden of Synonymous mutations

In [ ]:
## Count of CHIP

# summary(lm(INT_nCHIP ~ INT_nSynonymous, 
  #         data = baseVAR01qcd.aric_baseline_n_v05 ))

## Multivariate
# mod2_nCHIP <- lm(INT_nCHIP ~ INT_nSynonymous + 
 #           age_base + age_base_sqr + 
  #          Sex + race_BW  + ever_smoke + 
   #         INT_yang2012(bmi_base) + INT_yang2012(nonHDL_base) + INT_yang2012(hdl_base) +
    #        dm_126_base + htn_5_base + chd_is_base + 
    # chol_med_base + Center + v2_vs_other, 
      #      data = baseVAR01qcd.aric_baseline_n_v05 )

# summary(mod2_nCHIP)

###
# ggcoef_model(mod2_nCHIP, 
  #              include = dplyr::matches("INT_nSynonymous"),
   #             variable_labels = c("INT_nSynonymous" = "Passenger counts"), 
    #            show_p_values = FALSE, 
     #           signif_stars = FALSE,
      #          exponentiate = FALSE) +
# labs(title = "Passenger counts vs CHIP counts", x = "Beta")  + 
# scale_x_continuous(expand = c(0, 0), limits = c(0, 0.5))


In [ ]:
table(baseVAR01qcd.aric_baseline_n_v05$nCHIP_cat, exclude = NULL)

### Multinomial Logit Model

In [ ]:
## multinomial logit
# model1 <- multinom(nCHIP_cat ~ INT_nSynonymous, 
  #          data = baseVAR01qcd.aric_baseline_n_v05, Hess = FALSE )



# z_values_m1 <- summary(model1)$coefficients / summary(model1)$standard.errors

## multunomial probit
mod2.2_multinom_nCHIP <- multinom(nCHIP_cat ~ INT_nSynonymous + 
            age_base + age_base_sqr + 
            Sex + race_BW  + ever_smoke + 
            INT_yang2012(bmi_base) + INT_yang2012(nonHDL_base) + INT_yang2012(hdl_base) +
            dm_126_base + htn_5_base + chd_is_base + 
    chol_med_base + Center + v2_vs_other, 
            data = baseVAR01qcd.aric_baseline_n_v05, Hess = FALSE )

summary(mod2.2_multinom_nCHIP) 

# z_values_m2 <- summary(model2)$coefficients / summary(model2)$standard.errors

##
# z_values_m1
#z_values_m2

#2 * (1 - pnorm(abs(z_values_m1)))

#2 * (1 - pnorm(abs(z_values_m2)))

## Only keep variable of interest
# coef_subset_mod2 <- coef(model2)[, "INT_nSynonymous", drop = FALSE]


In [ ]:
summary(mod2.2_multinom_nCHIP)$coefficients

summary(mod2.2_multinom_nCHIP)$standard.errors

z_mod2.2_multinom_nCHIP <- summary(mod2.2_multinom_nCHIP)$coefficients/summary(mod2.2_multinom_nCHIP)$standard.errors

P_mod2.2_multinom_nCHIP <- 2 * (1 - pnorm(abs(z_mod2.2_multinom_nCHIP)))

# 2*Rmpfr::pnorm(Rmpfr::mpfr(z_mod2.2_multinom_nCHIP, precBits=10), lower.tail=FALSE)


In [ ]:
for(i in 1:4){
cat( gsub(pattern = ", ", replacement = ",", x = toString(
      c("multivariable", paste0(names(z_mod2.2_multinom_nCHIP[, 2])[i], " CHIP"), "INT_nSynonymous",
        summary(mod2.2_multinom_nCHIP)$coefficients[i, 2], 
        summary(mod2.2_multinom_nCHIP)$standard.errors[i, 2], 
        z_mod2.2_multinom_nCHIP[i, 2],
       P_mod2.2_multinom_nCHIP[i, 2]) ) ), "\n")
    }

In [ ]:
## Effect estimate
# for(i in 1:4){
# cat( gsub(pattern = ", ", replacement = ",", x = toString(
 #      c("multivariable", 
  #      paste0(names(z_mod2.2_multinom_nCHIP[, 2])[i], " CHIP"), 
   #     "INT_nSynonymous",
    #    summary(mod2.2_multinom_nCHIP)$coefficients[i, 2], 
    #    summary(mod2.2_multinom_nCHIP)$standard.errors[i, 2], 
     #   z_mod2.2_multinom_nCHIP[i, 2],
     #  P_mod2.2_multinom_nCHIP[i, 2]) ) ),
     # file = "passenger_vs_chip.multivariable_glm.20240604.csv", 
     #  append = T, fill = T)
# }

In [ ]:
     ######## addeded for case-control number 
      # Extract the data used in the model
model_multinom <- model.frame(mod2.2_multinom_nCHIP)


# Count the number of cases and controls
      table(model_multinom[[1]], exclude=NULL)
      
case_control_count_multinom <- table(model_multinom[[1]])

# Print number of cases and controls
cat("total N=", length(model_multinom[[1]]),"\n")
      
cat("nCHIP_cat: Number of controls:", case_control_count_multinom[1], "\n")

cat("nCHIP_cat: 1 clone:", case_control_count_multinom[2], "\n")

cat("nCHIP_cat: 2 clones:", case_control_count_multinom[3], "\n")

cat("nCHIP_cat: 3 clones:", case_control_count_multinom[4], "\n")

cat("nCHIP_cat: 4+ clones:", case_control_count_multinom[5], "\n")

for(i in 1:4){
cat( gsub(pattern = ", ", replacement = ",", x = toString(
      c("multivariable", 
        paste0(names(z_mod2.2_multinom_nCHIP[, 2])[i], " CHIP"), 
        "INT_nSynonymous",
        summary(mod2.2_multinom_nCHIP)$coefficients[i, 2], 
        summary(mod2.2_multinom_nCHIP)$standard.errors[i, 2], 
        z_mod2.2_multinom_nCHIP[i, 2],
       P_mod2.2_multinom_nCHIP[i, 2],
        
       case_control_count_multinom[i+1], 
       
        case_control_count_multinom[1], 
        
        length(model_multinom[[1]]) ) ) ),
    file = "passenger_vs_chip.multivariable_glm.2024Jul22.csv", 
       append = T, fill = T)

}
    


In [ ]:
# ggcoef_multinom(mod2.2_multinom_nCHIP, 
  #              include = dplyr::matches("INT_nSynonymous"),
   #             variable_labels = c("INT_nSynonymous" = "Passenger counts"), 
    #            show_p_values = FALSE, 
     #           signif_stars = FALSE,
      #          exponentiate = TRUE, 
       #         y.level_label = c(
        #        "1" = "1 CHIP", "2" = "2 CHIP", "3" = "3 CHIP", "4+" = "4+ CHIP")) +
# labs(title = "Passenger counts vs CHIP counts", x = "Multinomial Logit OR") + 
# scale_x_continuous(expand = c(0, 0), limits = c(0, 6))


## 3. Development of new clones, i.e. incident clones, vs burden of passenger counts

In [ ]:
# incident

summary(glm(incident_CH ~ INT_nSynonymous, 
            data = baseVAR01qcd.aric_baseline_n_v05, 
            family = "binomial"))

mod3_incidentCHIP <- (glm(incident_CH ~ INT_nSynonymous + 
                          age_base + age_base_sqr +
    Sex + race_BW + ever_smoke + 
            INT_yang2012(bmi_base) + INT_yang2012(nonHDL_base) + 
    INT_yang2012(hdl_base) + 
            dm_126_base + htn_5_base + chd_is_base + 
    chol_med_base + Center + v2_vs_other, 
            data = baseVAR01qcd.aric_baseline_n_v05, 
            family = "binomial"))

summary(mod3_incidentCHIP)

ggcoef_model(mod3_incidentCHIP, 
                include = dplyr::matches("INT_nSynonymous"),
                variable_labels = c("INT_nSynonymous" = "Passenger counts"), 
                show_p_values = FALSE, 
                signif_stars = FALSE,
                exponentiate = TRUE) +
labs(title = "Passenger counts vs incident CHIP", x = "OR") + 
scale_x_continuous(expand = c(0, 0), limits = c(0.5, 1.5))



## 4. delta VAF vs burden of Synonymous mutations

In [ ]:
cor(baseVAR01qcd.aric_baseline_n_v05$CHIP_expanded_vs_noChange, 
    baseVAR01qcd.aric_baseline_n_v05$nSynonymous, 
    use="complete", method="pearson")

cor(baseVAR01qcd.aric_baseline_n_v05$CHIP_expanded_vs_noChange, 
    baseVAR01qcd.aric_baseline_n_v05$INT_nSynonymous, 
    use="complete", method="pearson")

In [ ]:
## Logistic regression
mod4_growing <- (baseVAR01qcd.aric_baseline_n_v05 %>% 
glm(CHIP_expanded_vs_noChange ~ INT_nSynonymous + 
    age_base + age_base_sqr +
    Sex + race_BW + ever_smoke + 
            INT_yang2012(bmi_base) + INT_yang2012(nonHDL_base) + 
    INT_yang2012(hdl_base) + 
            dm_126_base + htn_5_base + chd_is_base + 
    chol_med_base + Center + v2_vs_other, 
            data =  ., family="binomial"))


summary(mod4_growing)

ggcoef_model(mod4_growing, 
                include = dplyr::matches("INT_nSynonymous"),
                variable_labels = c("INT_nSynonymous" = "Passenger counts"), 
                show_p_values = FALSE, 
                signif_stars = FALSE,
                exponentiate = TRUE) +
labs(title = "Passenger counts vs Clonal expansion", x = "OR") + 
scale_x_continuous(expand = c(0, 0), limits = c(0.5, 1.5))


In [ ]:
ls()

In [ ]:
pdf("passenger_counts.n1_4.pdf",
    width = 8, height= 4)
##
ggcoef_model(mod1_anychip, 
                include = dplyr::matches("INT_nSynonymous"),
                variable_labels = c("INT_nSynonymous" = "Passenger counts"), 
                show_p_values = FALSE, 
                signif_stars = FALSE,
                exponentiate = TRUE) +
labs(title = "Passenger counts vs any CHIP", x = "OR") + 
scale_x_continuous(expand = c(0, 0), limits = c(0.5, 1.5))

## CHIP counts
ggcoef_model(mod2_nCHIP, 
                include = dplyr::matches("INT_nSynonymous"),
                variable_labels = c("INT_nSynonymous" = "Passenger counts"), 
                show_p_values = FALSE, 
                signif_stars = FALSE,
                exponentiate = FALSE) +
labs(title = "Passenger counts vs CHIP counts", x = "Beta")  + 
scale_x_continuous(expand = c(0, 0), limits = c(0, 0.5))

ggcoef_multinom(mod2.2_multinom_nCHIP, 
                include = dplyr::matches("INT_nSynonymous"),
                variable_labels = c("INT_nSynonymous" = "Passenger counts"), 
                show_p_values = FALSE, 
                signif_stars = FALSE,
                exponentiate = TRUE, 
                y.level_label = c(
                "1" = "1 CHIP", "2" = "2 CHIP", "3" = "3 CHIP", "4+" = "4+ CHIP")) +
labs(title = "Passenger counts vs CHIP counts", x = "Multinomial Logit OR") + 
scale_x_continuous(expand = c(0, 0), limits = c(0, 4))


## incident clone
ggcoef_model(mod3_incidentCHIP, 
                include = dplyr::matches("INT_nSynonymous"),
                variable_labels = c("INT_nSynonymous" = "Passenger counts"), 
                show_p_values = FALSE, 
                signif_stars = FALSE,
                exponentiate = TRUE) +
labs(title = "Passenger counts vs incident CHIP", x = "OR") + 
scale_x_continuous(expand = c(0, 0), limits = c(0.5, 1.5))


## Clone status
ggcoef_model(mod4_growing, 
                include = dplyr::matches("INT_nSynonymous"),
                variable_labels = c("INT_nSynonymous" = "Passenger counts"), 
                show_p_values = FALSE, 
                signif_stars = FALSE,
                exponentiate = TRUE) +
labs(title = "Passenger counts vs Clonal expansion", x = "OR") + 
scale_x_continuous(expand = c(0, 0), limits = c(0.5, 1.5))



ggcoef_multinom(mod4_multinomial, 
                include = dplyr::matches("INT_nSynonymous"),
                variable_labels = c("INT_nSynonymous" = "Passenger counts"), 
                show_p_values = FALSE, 
                signif_stars = TRUE,
                exponentiate = TRUE, 
                add_reference_rows = FALSE, 
                y.level_label = c("1" = "Clonal expansion", "2" = "Clonal regression"), 
                categorical_terms_pattern = "{level} (ref: {reference_level})"     )  +
labs(title = "Passenger counts vs CHIP clone status", x = "Multinomial Logit OR") + 
scale_x_continuous(expand = c(0, 0), limits = c(0, 3))

dev.off()

In [ ]:
## Date: May 30, 2024
# save.image(file="/medpop/esp2/mesbah/projects/ch_progression/aric/passenger/data_n_models.passenger_hitchhiker_20240529.rda")
# June 04, 2024
#save.image(file="/medpop/esp2/mesbah/projects/ch_progression/aric/passenger/data_n_models.passenger_hitchhiker_20240604.rda")

In [ ]:
# load("/medpop/esp2/mesbah/projects/ch_progression/aric/passenger/data_n_models.passenger_hitchhiker_20240529.rda")
load("/medpop/esp2/mesbah/projects/ch_progression/aric/passenger/data_n_models.passenger_hitchhiker_20240604.rda")

## Mutational Signature Analysis

In [ ]:
## Signature analyses
####
ref_genome <- "BSgenome.Hsapiens.UCSC.hg38"
library(ref_genome, character.only = TRUE)
library("MutationalPatterns")
# options(stringsAsFactors = F)
# library(data.table)
library(GenomicRanges)
library(Rsamtools)
library(MASS)
library(VariantAnnotation)
####
###################


In [ ]:
## Mutation types:
mut_type <- function (nuc_subs) {
    muts <- nuc_subs
    types <- unlist(muts)
    types <- gsub("G>T", "C>A", types)
    types <- gsub("G>C", "C>G", types)
    types <- gsub("G>A", "C>T", types)
    types <- gsub("A>T", "T>A", types)
    types <- gsub("A>G", "T>C", types)
    types <- gsub("A>C", "T>G", types)
    return(types)
}

In [ ]:
# BiocManager::install("BSgenome.Hsapiens.UCSC.hg38")
# BiocManager::install("MutationalPatterns")
ref_genome <- "BSgenome.Hsapiens.UCSC.hg38"

library(ref_genome, character.only = TRUE)

library("MutationalPatterns")
options(stringsAsFactors = F)
library(data.table)
library(GenomicRanges)
library(Rsamtools)
library(MASS)
library(VariantAnnotation)

Spectrum / signature analysis using R MutationalPatterns
read_vcfs_as_granges()
mut_matrix()
mut_type_occurrences()
plot_spectrum()
Can discuss the next steps after spectrum is created with me/Zheming once there and when you decide if you want to use MutationalPatterns signature fitting, NMF de-novo, or MuSiCal mvNMF

In [ ]:
head(synon_base.qcd)

In [ ]:
synon_base.qcd$context <- as.character(Biostrings::getSeq(BSgenome::getBSgenome(ref_genome), 
        synon_base.qcd$CHROM, synon_base.qcd$POS - 1, synon_base.qcd$POS + 1))
head(synon_base.qcd)

In [ ]:
synon_base.qcd$mut_type <- mut_type(synon_base.qcd$nuc_subs)
head(synon_base.qcd)

In [ ]:
## correct sequence context

synon_base.qcd$mut_context <- synon_base.qcd$context 
head(synon_base.qcd)

x <- which(synon_base.qcd$nuc_subs != synon_base.qcd$mut_type)
y <- synon_base.qcd$context[x]
y <- IRanges::reverse(chartr("ATGC", "TACG", y))

synon_base.qcd$mut_context[x] <- y
head(synon_base.qcd)

In [ ]:
# Step 1: Install and load the necessary packages
# install.packages("deconstructSigs")
library(deconstructSigs)

# Step 2: Load your data
# Replace this with the actual path to your data
# data_path <- "path_to_your_data.csv"
# data <- read.csv(data_path)

# Step 3: Convert your data into the format required for signature analysis
input <- mut.to.sigs.input(mut.ref = synon_base.qcd, 
                           sample.id = "Sample_ID",
                           chr = "CHROM",
                           pos = "POS",
                           ref = "REF",
                           alt = "ALT", 
                           bsg = BSgenome.Hsapiens.UCSC.hg38::BSgenome.Hsapiens.UCSC.hg38)

# Step 4: Determine the contribution of known signatures to your data
signatures <- whichSignatures(tumor.ref = input, 
                              signatures.ref = signatures.cosmic,)

# Step 5: Plot the results
plotSignatures(signatures)

In [ ]:
sort(colSums(input))

table(rowSums(input)>=50)

In [ ]:

(table(synon_base$FILTER))

table(table(synon_base$Sample_ID[synon_base$FILTER %in% c("PASS", "weak_evidence")]))

In [ ]:
  # 3. Counting subs
  freqs = table(paste(mutations$sub,paste(substr(mutations$trinuc_ref_py,1,1),substr(mutations$trinuc_ref_py,3,3),sep="-"),sep=","))
  sub_vec = c("C>A","C>G","C>T","T>A","T>C","T>G")
  ctx_vec = paste(rep(c("A","C","G","T"),each=4),rep(c("A","C","G","T"),times=4),sep="-")
  full_vec = paste(rep(sub_vec,each=16),rep(ctx_vec,times=6),sep=",")
  freqs_full = freqs[full_vec]; freqs_full[is.na(freqs_full)] = 0; names(freqs_full) = full_vec
  
  xstr = paste(substr(full_vec,5,5), substr(full_vec,1,1), substr(full_vec,7,7), sep="")
  
  if(is.null(save)){dev.new(width=12,height=4)}
  colvec = rep(c("dodgerblue","black","red","grey70","olivedrab3","plum2"),each=16)
  y = freqs_full; maxy = max(y)
  h = barplot(y, las=2, col=colvec, border=NA, ylim=c(0,maxy*1.5), space=1, cex.names=0.6, names.arg=xstr, ylab="Number of mutations", main=paste0("Total number of mutations: ",nrow(mutations),add_to_title))
  for (j in 1:length(sub_vec)) {
    xpos = h[c((j-1)*16+1,j*16)]
    rect(xpos[1]-0.5, maxy*1.2, xpos[2]+0.5, maxy*1.3, border=NA, col=colvec[j*16])
    text(x=mean(xpos), y=maxy*1.3, pos=3, label=sub_vec[j])
  } 
  if(!is.null(save)){ 
    dev.copy(pdf,save,width=12,height=4); dev.off()}


In [ ]:
ls()

## Growth rate

In [ ]:
## June 04, 2024

load("/medpop/esp2/mesbah/projects/ch_progression/aric/passenger/data_n_models.passenger_hitchhiker_20240604.rda")



In [ ]:
ls()

#### qc data ####
## synon_base.qcd_pass_auto
synon_base.qcd_filtered <- synon_base.qcd %>% 
  filter(FILTER == "PASS" & 
           nchar(REF)==1 & 
           nchar(ALT)==1 & 
           !(CHROM%in%c("chrY", "chrX")) &
           AD_ALT>=3 & VAF<=0.25 &
           (varID %in% nVar01qcd.synon_base$Var1[nVar01qcd.synon_base$Freq==1])
  )

nrow(synon_base.qcd_filtered)

In [ ]:
# save(synon_base.qcd_filtered, 
  #   aric_baseline_n_v05.clones, 
   #  file="/medpop/esp2/mesbah/projects/ch_progression/aric/passenger/synon_base.qcd_filtered.202406.rda")


In [ ]:
cln_grt.vaf2.DP20_base.corrected_ordered$VAF.v2_v2 <- cln_grt.vaf2.DP20_base.corrected_ordered$VAF.v2

cln_grt.vaf2.DP20_base.corrected_ordered$VAF.v2_v2[cln_grt.vaf2.DP20_base.corrected_ordered$VAF.v2_v2==1e-4] <- 0.001

summary(cln_grt.vaf2.DP20_base.corrected_ordered$VAF.v2)

summary(cln_grt.vaf2.DP20_base.corrected_ordered$VAF.v2_v2)

cln_grt.vaf2.DP20_base.corrected_ordered$logdVAF_by_dT_ver2 <- log(cln_grt.vaf2.DP20_base.corrected_ordered$VAF.v5/cln_grt.vaf2.DP20_base.corrected_ordered$VAF.v2_v2)/cln_grt.vaf2.DP20_base.corrected_ordered$dAge

cln_grt.vaf2.DP20_base.corrected_ordered$dVAF_by_dT_ver2 <- (cln_grt.vaf2.DP20_base.corrected_ordered$VAF.v5 - cln_grt.vaf2.DP20_base.corrected_ordered$VAF.v2_v2)/cln_grt.vaf2.DP20_base.corrected_ordered$dAge


In [ ]:

cor(cln_grt.vaf2.DP20_base.corrected_ordered$dVAF_by_dT_ver2, 
    cln_grt.vaf2.DP20_base.corrected_ordered$logdVAF_by_dT_ver2, use="complete")

plot(cln_grt.vaf2.DP20_base.corrected_ordered$dVAF_by_dT_ver2, 
     cln_grt.vaf2.DP20_base.corrected_ordered$logdVAF_by_dT_ver2)

In [ ]:
######### continous change in vaf

syno.cln_grt.v1 <- merge(cln_grt.vaf2.DP20_base.corrected_ordered, 
                         synon_base.qcd_pass_auto, 
                         by.x="GWAS_ID.x", 
                         by.y="gwasid")

nrow(syno.cln_grt.v1)


In [ ]:
names(syno.cln_grt.v1)

In [ ]:
# Passenger count
cor.test(syno.cln_grt.v1$logdVAF_by_dT_ver2, syno.cln_grt.v1$nSynonymous)

cor.test(syno.cln_grt.v1$dVAF_by_dT_ver2, syno.cln_grt.v1$nSynonymous)



# Supplementary Table 5. Two-sided Pearson's correlation tests between the growth rate of CHIP clones and synonymous variant allele fraction (VAF).

## Log Growth rate, LogdVAF_by_dT = log(VAFFollow-up/VAFBaseline) / (AgeFollow-up - AgeBaseline)

In [ ]:
length(syno.cln_grt.v1$logdVAF_by_dT_ver2)
length(syno.cln_grt.v1$minVAF)
length(unique(syno.cln_grt.v1$ARIC_ID))

In [ ]:
# VAF Passenger 

# Max VAF
cat("max vaf\n")
cor.test(syno.cln_grt.v1$logdVAF_by_dT_ver2, (syno.cln_grt.v1$maxVAF) )

# Minimum VAF

cat("min vaf\n")
cor.test(syno.cln_grt.v1$logdVAF_by_dT_ver2, (syno.cln_grt.v1$minVAF) )

# Avg. VAF

cat("avg. vaf\n")
cor.test(syno.cln_grt.v1$logdVAF_by_dT_ver2, (syno.cln_grt.v1$avgVAF) )

# Median VAF

cat("median vaf\n")
cor.test(syno.cln_grt.v1$logdVAF_by_dT_ver2, (syno.cln_grt.v1$medianVAF) )



## Growth rate, dVAF_by_dT =  (VAFFollow-up - VAFBaseline) / (AgeFollow-up - AgeBaseline)

In [ ]:
length(syno.cln_grt.v1$dVAF_by_dT_ver2)
length(syno.cln_grt.v1$minVAF)
length(unique(syno.cln_grt.v1$ARIC_ID))


In [ ]:
# Min VAF vs dVAFdT
cor.test(syno.cln_grt.v1$dVAF_by_dT_ver2, syno.cln_grt.v1$minVAF) 

# Avg. VAF vs dVAFdT
cor.test(syno.cln_grt.v1$dVAF_by_dT_ver2, syno.cln_grt.v1$avgVAF) 

# Median VAF vs dVAFdT
cor.test(syno.cln_grt.v1$dVAF_by_dT_ver2, syno.cln_grt.v1$medianVAF)

# Max VAF vs dVAFdT
cor.test(syno.cln_grt.v1$dVAF_by_dT_ver2, syno.cln_grt.v1$maxVAF )

# Supplementary Table 6. Association between synonymous variant allele fraction (VAF) and growth rate of CHIP clones

## Linear regression model: LogdVAF_by_dT ~ Synonymous mutation VAF + Synonymous counts + covariates
* Log Growth rate, LogdVAF_by_dT = log(VAFFollow-up/VAFBaseline) / (AgeFollow-up - AgeBaseline)

### Minimum VAF

In [ ]:
cat("min VAF\n")

logdvaf_by_dt_minVAF <- (syno.cln_grt.v1 %>% 
          lm((logdVAF_by_dT_ver2) ~ 
               minVAF + nSynonymous + 
             Gene_Group + Variant_type + Sex  + race_BW + 
           ever_smoke + INT_yang2012(hdl_base)  +  
           INT_yang2012(nonHDL_base) + INT_yang2012(bmi_base) + 
           dm_126_base + htn_5_base + 
                    chd_is_base + 
           age_base + dAge +   chol_med_base + 
             
             log(maxDP) + log(DP.v2) +  v2_vs_other + 
           center + Imputed_VAF_v2 + is_notMUTECT+ is_notHiSeq, 
             data=.))
  

summary(logdvaf_by_dt_minVAF)  


## 
logdvaf_by_dt_minVAF_data <- model.frame(logdvaf_by_dt_minVAF)

logdvaf_by_dt_minVAF_data

length(logdvaf_by_dt_minVAF_data[[1]])


### Median VAF

In [ ]:

cat("median VAF\n")

logdvaf_by_dt_medianVAF <- (syno.cln_grt.v1 %>% 
          lm((logdVAF_by_dT_ver2) ~ 
               medianVAF + nSynonymous + 
             Gene_Group + Variant_type + Sex  + race_BW + 
           ever_smoke + INT_yang2012(hdl_base)  +  
           INT_yang2012(nonHDL_base) + INT_yang2012(bmi_base) + 
           dm_126_base + htn_5_base + 
                    chd_is_base + 
           age_base + dAge +   chol_med_base + 
             
             log(maxDP) + 
           log(DP.v2) +  v2_vs_other + 
           center + Imputed_VAF_v2 + is_notMUTECT+ is_notHiSeq, 
             data=.))


summary(logdvaf_by_dt_medianVAF)

## 
logdvaf_by_dt_medianVAF_data <- model.frame(logdvaf_by_dt_medianVAF)

logdvaf_by_dt_medianVAF_data

length(logdvaf_by_dt_medianVAF_data[[1]])



### Regression analysis

In [ ]:
cat("min VAF\n")

minVAF_dvafdT <- (syno.cln_grt.v1 %>% 
          lm(dVAF_by_dT_ver2 ~ 
               minVAF + nSynonymous + 
             Gene_Group + Variant_type + Sex  + race_BW + 
           ever_smoke + INT_yang2012(hdl_base)  +  
           INT_yang2012(nonHDL_base) + INT_yang2012(bmi_base) + 
           dm_126_base + htn_5_base + 
                    chd_is_base + 
           age_base + dAge +   chol_med_base + 
             
             log(maxDP) + 
           log(DP.v2) +  v2_vs_other + 
           center + Imputed_VAF_v2 + is_notMUTECT+ is_notHiSeq, 
             data=.))

summary(minVAF_dvafdT)  

length(model.frame(minVAF_dvafdT)[[1]])

cat("median VAF\n")

medianVAF_dvafdT <- (syno.cln_grt.v1 %>% 
          lm(dVAF_by_dT_ver2 ~ 
               medianVAF + nSynonymous + 
             Gene_Group + Variant_type + Sex  + race_BW + 
           ever_smoke + INT_yang2012(hdl_base)  +  
           INT_yang2012(nonHDL_base) + INT_yang2012(bmi_base) + 
           dm_126_base + htn_5_base + 
                    chd_is_base + 
           age_base + dAge +   chol_med_base + 
             
             log(maxDP) + 
           log(DP.v2) +  v2_vs_other + 
           center + Imputed_VAF_v2 + is_notMUTECT+ is_notHiSeq, 
             data=.))

summary(medianVAF_dvafdT)

length(model.frame(medianVAF_dvafdT)[[1]])

In [ ]:
# Create residual plots
par(mfrow = c(2, 2))  # Set up the graphics window to show 4 plots
plot(minVAF_dvafdT)  # Create the plots

plot(medianVAF_dvafdT)  # Create the plots

